# ML-SQL language (1st-take)

## Authors

Written by: Neeraj Asthana (under Professor Robert Brunner)

University of Illinois at Urbana-Champaign

Summer 2016

## Acknowledgements

Followed Tutorial at: http://www.onlamp.com/lpt/a/6435

## Description

This notebook is meant to experiment with constructs for the ML-SQL language. The goal is to be able to understand ML-SQL syntax and port commands to actionable directives in Python.

___

In [205]:
#Libraries
#from pyparsing import Word, Literal, alphas, Optional, OneOrMore, Group, Or, Combine, oneOf
from pyparsing import *
import string
import sys
import pandas as pd
from sklearn import svm
from sklearn.cross_validation import train_test_split

___

### Grammer Definition

Literals and Valid Symbols that are possible in the ML-SQL language

In [121]:
letters = string.ascii_letters
punctuation = string.punctuation
numbers = string.digits
whitespace = string.whitespace

#combinations
everything = letters + punctuation + numbers
everythingWOQuotes = everything.replace("\"", "").replace("'", "")

#Booleans
bools = Literal("True") + Literal("False")

#Parenthesis and Quotes
openParen = Literal("(").suppress()
closeParen = Literal(")").suppress()
Quote = Literal('"').suppress()

#includes every combination except whitespace
everything

'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~0123456789'

___

### READ

In [137]:
filename = Word(everything).setResultsName("filename")

#define so that there can be multiple verisions of READ
readKeyword = oneOf(["Read", "READ"]).suppress()

#Define Read Optionals
#header
headerLiteral = (Literal("header") + Literal("=")).suppress()
header = Optional(headerLiteral + Or(bools).setResultsName("header"), default = "False" )

#separator
separatorLiteral = (Or([Literal("sep"), Literal("separator")]) + Literal("=")).suppress()
definesep = Quote + Word(everythingWOQuotes + whitespace).setResultsName("sep") + Quote
separator = Optional(separatorLiteral + definesep, default = ",")

#Compose Read Optionals
readOptions = Optional(openParen + separator + header + closeParen)

read = readKeyword + filename + readOptions

In [149]:
readTest = 'READ /home/ubuntu/notebooks/ML-SQL/Classification/iris.data (sep="," header=False)'

readTestResult = read.parseString(readTest)

filename = readTestResult.filename
header = readTestResult.header
sep = readTestResult.sep

#Function to lower a string value of "True" or "False" to an actual python boolean value
def str_to_bool(s):
    if s == 'True':
         return True
    elif s == 'False':
         return None
    else:
         raise ValueError ("Cannot lower value " + s + " to a boolean value")
            
#read parameters from parsed statement and read the file
f = pd.read_csv(filename, sep = sep, header = str_to_bool(header))
f.head()

,0,1,2,3,4
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


___

### SPLIT

Splits dataset into training, testing, and validation sets. Give 3 non-negative decimals that sum to 1 to specify these quantities.

In [165]:
#define so that there can be multiple verisions of Split
splitKeyword = oneOf(["Split", "SPLIT"]).suppress()

#Phrases used to organize splits
trainPhrase = (Literal("train") + Literal("=")).suppress()
testPhrase = (Literal("test") + Literal("=")).suppress()
valPhrase = (Literal("validation") + Literal("=")).suppress()

#train, test, validation split values
trainS = Combine(Literal(".") + Word(numbers)).setResultsName("train_split")
testS = Combine(Literal(".") + Word(numbers)).setResultsName("test_split")
valS = Combine(Literal(".") + Word(numbers)).setResultsName("validation_split")

#Compose phrases and values together 
training = trainPhrase + trainS
testing = testPhrase + testS
val = valPhrase + valS

#Creating Optional Split phrase
ocomma = Optional(",").suppress()
split = Optional(splitKeyword + openParen + training + ocomma + testing + ocomma + val + closeParen)

#Combining READ and SPLIT keywords into one clause for combined use
read_split = read + split

In [207]:
#Split test
splitTest = "SPLIT (train = .8, test = .2, validation = .0)"

print(split.parseString(splitTest))

['.8', '.2', '.0']


In [168]:
#Read with Split test
read_split_test = readTest + " "+ splitTest

print(read_split.parseString(read_split_test))

['/home/ubuntu/notebooks/ML-SQL/Classification/iris.data', ',', 'False', '.6', '.2', '.2']


___

### Classify

Define an algorithm to perform a classification task on 

Supported classifier: SVM, Logistic Regression

In [200]:
#define so that there can be multiple verisions of Classify
classifyKeyword = oneOf(["Classify", "CLASSIFY"]).suppress()

#Phrases to organize predictor and label column numbers
predPhrase = (Literal("predictors") + Literal("=")).suppress()
labelPhrase = (Literal("label") + Literal("=")).suppress()

#define predictor and label column numbers
predictorsDef = OneOrMore(Word(numbers) + ocomma).setResultsName("predictors")
labelDef = Word(numbers).setResultsName("label")

#combine phrases with found column numbers
preds = predPhrase + openParen + predictorsDef + closeParen
labels = labelPhrase + labelDef

classify = Optional(classifyKeyword + openParen + preds + ocomma + labels + closeParen)

In [202]:
classifyTest = "CLASSIFY (predictors = (1,2,3,4), label = 5)"

print(classify.parseString(classifyTest))

['1', '2', '3', '4', '5']


___

## Examples

I include the 3 words: READ, SPLIT, CLASSIFY and show a basic example of a classification task on the Iris-data set

In [208]:
read_split_classify = read + split + classify

query1 = readTest + " " + splitTest + " " + classifyTest

print(query1)

READ /home/ubuntu/notebooks/ML-SQL/Classification/iris.data (sep="," header=False) SPLIT (train = .8, test = .2, validation = .0) CLASSIFY (predictors = (1,2,3,4), label = 5)


In [210]:
#define a pipeline to accomplish all of the data tasks we envision
result1 = read_split_classify.parseString(query1)

#Extract relevant features from the query
filename1 = result1.filename
header1 = result1.header
sep1 = result1.sep
train1 = result1.train_split
test1 = result1.test_split
predictors1 = result1.predictors
label1 = result1.label

In [224]:
#Preform classification dataflow

#read file 
file1 = pd.read_csv(filename1, header = str_to_bool(header1), sep = sep1)

#predictors and labels
pred_cols = map(int, predictors1)
pred_cols = map(lambda x: x - 1, pred_cols)
label_col = int(label1) - 1

X = file1.ix[:,pred_cols]
y = file1.ix[:,label_col]

#Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=float(train1), test_size=float(test1))

#Train SVM
clf = svm.SVC()
clf.fit(X_train, y_train)

#Performance on test data
clf.score(X_test, y_test)

0.93333333333333335